In [ ]:
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 26.5 MB/s 
     |████████████████████████████████| 771 kB 61.1 MB/s 
     |████████████████████████████████| 310 kB 73.8 MB/s 
     |████████████████████████████████| 61 kB 10.9 MB/s 
     |████████████████████████████████| 662 kB 73.0 MB/s 
     |████████████████████████████████| 14.4 MB 54.8 MB/s 
     |████████████████████████████████| 5.8 MB 64.4 MB/s 
     |████████████████████████████████| 122 kB 68.7 MB/s 
     |████████████████████████████████| 355 kB 63.8 MB/s 
     |████████████████████████████████| 398 kB 69.3 MB/s 
     |████████████████████████████████| 7.6 MB 72.2 MB/s 
     |████████████████████████████████| 182 kB 74.7 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=746185 sha256=d0ddeacdec70e58f87d566a18b7c494f4800d3f0ac40673c5a9ed2ecff937c82
  Stored in directory: /root/.cache/pip/whe

#Load Data

In [ ]:
!pip install gdown


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct

Downloading...
From: https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
To: /content/data.zip
100% 4.16M/4.16M [00:00<00:00, 255MB/s]


In [ ]:
!unzip -o "data.zip"  -d  "/content"

Archive:  data.zip
  inflating: /content/amazon_min.csv  
  inflating: /content/book_min.csv   
  inflating: /content/ciao_min.csv   
  inflating: /content/ecom_min.csv   
  inflating: /content/food_min.csv   
  inflating: /content/movies_min.csv  


In [ ]:

movie_10k_df = pd.read_csv("movies_min.csv")
amazon_df =pd.read_csv('amazon_min.csv')
book_df=pd.read_csv("book_min.csv")
food_df = pd.read_csv('food_min.csv')
ecommerce_df=pd.read_csv('ecom_min.csv')
ciao_df=pd.read_csv('ciao_min.csv')

# Details about dataset

In [ ]:
summary_df=pd.DataFrame(columns=['Dataset','Algorithm','Sparsity','Precision@k'])

In [ ]:
userID="userID"
itemID="itemID"
score="rating"
timestamp='time'

# Sparsity 

In [ ]:
from scipy.sparse import csr_matrix
def checkSparsity(main_df):
   
    # pivot_df = main_df.pivot_table(
    #     index='itemID',
    #     columns='userID',
    #     values='rating'
    # )
    totalUsers=main_df.userID.unique().shape[0]
    totalitems= main_df.itemID.unique().shape[0]
    total=totalUsers*totalitems
    totalreconds=main_df.shape[0]
    sparsity=(total-totalreconds)/total
    return sparsity

In [ ]:

sparsity_food=checkSparsity(food_df)
sparsity_food

0.9758585086369416

In [ ]:
sparsity_amazon=checkSparsity(amazon_df)
sparsity_amazon

0.9919133172746379

In [ ]:

sparsity_movielens=checkSparsity(movie_10k_df)
sparsity_movielens

0.900792867888234

In [ ]:
sparsity_ecom=checkSparsity(ecommerce_df)
sparsity_ecom

0.9994459394919134

<!-- Observations :
1. Very sparse dataset sparsity 0.98
2. Many items and users in the dataset has less interaction data -->

<!-- ### Divide dataset based on following:
- All users rated very few items - will use contant based filtering method.To get threshold value for filtering we first find median of number of rating given by each users.Next choose value less then that median value such that we decrese sparsity as much as possible(less then .95)
- For other data we will go with collaborative filtering or other deep learning methods -->

In [ ]:
sparsity_book=checkSparsity(book_df)
sparsity_book

0.9911869365436147

In [ ]:
sparsity_ciao=checkSparsity(ciao_df)
sparsity_ciao

0.9914865451250656

In [ ]:
# movie_10k_df['rating']=1
# amazon_df['rating']=1
# book_df['rating']=1
# food_df['rating']=1
# ciao_df['rating']=1
# ecommerce_df['rating']=1

In [ ]:
!pip install papermill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scrapbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Run BiVAE

In [ ]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split,python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
PyTorch version: 1.13.0+cu116
Cornac version: 1.14.2


In [ ]:


# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [ ]:
data =movie_10k_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"MovieLens",'Algorithm':"BiVAE",'Sparsity':sparsity_movielens,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 943
Number of items: 1032


  0%|          | 0/500 [00:00<?, ?it/s]

Took 46.4921 seconds for training.
Took 0.8553 seconds for prediction.
MAP:	0.153590
NDCG:	0.480573
Precision@K:	0.424496
Recall@K:	0.237295


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295


In [ ]:
data =book_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Book Dataset",'Algorithm':"BiVAE",'Sparsity':sparsity_book,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 1498
Number of items: 3530


  0%|          | 0/500 [00:00<?, ?it/s]

Took 102.6556 seconds for training.
Took 3.9834 seconds for prediction.
MAP:	0.032110
NDCG:	0.071512
Precision@K:	0.047196
Recall@K:	0.067536


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536


In [ ]:
data =food_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Food.com",'Algorithm':"BiVAE",'Sparsity':sparsity_food,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 1926
Number of items: 1577


  0%|          | 0/500 [00:00<?, ?it/s]

Took 75.6618 seconds for training.
Took 2.4432 seconds for prediction.
MAP:	0.015620
NDCG:	0.046031
Precision@K:	0.035045
Recall@K:	0.039292


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292


In [ ]:
data =amazon_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Amazon Product",'Algorithm':"BiVAE",'Sparsity':sparsity_amazon,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 2952
Number of items: 2771


  0%|          | 0/500 [00:00<?, ?it/s]

Took 128.6497 seconds for training.
Took 5.0802 seconds for prediction.
MAP:	0.019607
NDCG:	0.044613
Precision@K:	0.027479
Recall@K:	0.047853


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853


In [ ]:
data =ciao_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Ciao",'Algorithm':"BiVAE",'Sparsity':sparsity_ciao,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 2037
Number of items: 4389


/usr/local/lib/python3.8/dist-packages/cornac/data/dataset.py:361: UserWarning: 482 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


  0%|          | 0/500 [00:00<?, ?it/s]

Took 132.8713 seconds for training.
Took 7.4279 seconds for prediction.
MAP:	0.050224
NDCG:	0.121652
Precision@K:	0.089396
Recall@K:	0.103509


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853
4,Ciao,BiVAE,0.991487,0.089396,0.103509


In [ ]:
summary_df[['Dataset','Algorithm','Precision@k']]

,Dataset,Algorithm,Precision@k
0,MovieLens,BiVAE,0.424496
1,Book Dataset,BiVAE,0.047196
2,Food.com,BiVAE,0.035045
3,Amazon Product,BiVAE,0.027479
4,Ciao,BiVAE,0.089396


# VAE

In [ ]:


# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [ ]:
data =movie_10k_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.VAECF(
    k=LATENT_DIM,
    autoencoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)


Number of users: 943
Number of items: 1032


In [ ]:

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')


  0%|          | 0/500 [00:00<?, ?it/s]

Took 17.3702 seconds for training.
Took 1.2654 seconds for prediction.
MAP:	0.137548
NDCG:	0.440285
Precision@K:	0.386426
Recall@K:	0.217139


In [ ]:
df2 = {'Dataset':"MovieLens",'Algorithm':"VAE",'Sparsity':sparsity_movielens,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853
4,Ciao,BiVAE,0.991487,0.089396,0.103509
5,MovieLens,VAE,0.900793,0.386426,0.217139


In [ ]:
data =book_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.VAECF(
    k=LATENT_DIM,
    autoencoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Book Dataset",'Algorithm':"VAE",'Sparsity':sparsity_book,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 1498
Number of items: 3530


  0%|          | 0/500 [00:00<?, ?it/s]

Took 35.1625 seconds for training.
Took 4.8108 seconds for prediction.
MAP:	0.014607
NDCG:	0.040234
Precision@K:	0.028905
Recall@K:	0.040259


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853
4,Ciao,BiVAE,0.991487,0.089396,0.103509
5,MovieLens,VAE,0.900793,0.386426,0.217139
6,Book Dataset,VAE,0.991187,0.028905,0.040259


In [ ]:
data =food_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.VAECF(
    k=LATENT_DIM,
    autoencoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Food.com",'Algorithm':"VAE",'Sparsity':sparsity_food,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 1926
Number of items: 1577


  0%|          | 0/500 [00:00<?, ?it/s]

Took 40.4806 seconds for training.
Took 4.8920 seconds for prediction.
MAP:	0.017974
NDCG:	0.055201
Precision@K:	0.042745
Recall@K:	0.045760


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853
4,Ciao,BiVAE,0.991487,0.089396,0.103509
5,MovieLens,VAE,0.900793,0.386426,0.217139
6,Book Dataset,VAE,0.991187,0.028905,0.040259
7,Food.com,VAE,0.975859,0.042745,0.045760


In [ ]:
data =ciao_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.VAECF(
    k=LATENT_DIM,
    autoencoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Ciao",'Algorithm':"VAE",'Sparsity':sparsity_ciao,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 2037
Number of items: 4389


/usr/local/lib/python3.8/dist-packages/cornac/data/dataset.py:361: UserWarning: 482 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


  0%|          | 0/500 [00:00<?, ?it/s]

Took 43.9169 seconds for training.
Took 8.6590 seconds for prediction.
MAP:	0.050924
NDCG:	0.121093
Precision@K:	0.089494
Recall@K:	0.102869


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853
4,Ciao,BiVAE,0.991487,0.089396,0.103509
5,MovieLens,VAE,0.900793,0.386426,0.217139
6,Book Dataset,VAE,0.991187,0.028905,0.040259
7,Food.com,VAE,0.975859,0.042745,0.045760
8,Ciao,VAE,0.991487,0.089494,0.102869


In [ ]:
data =amazon_df.copy()
data=data[['userID','itemID','rating']]
train, test = python_stratified_split(data, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))
bivae = cornac.models.VAECF(
    k=LATENT_DIM,
    autoencoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
df2 = {'Dataset':"Amazon Product",'Algorithm':"VAE",'Sparsity':sparsity_amazon,'Precision@k':eval_precision,'Recall@k':eval_recall}
summary_df = summary_df.append(df2, ignore_index = True)
summary_df

Number of users: 2952
Number of items: 2771


  0%|          | 0/500 [00:00<?, ?it/s]

Took 60.4917 seconds for training.
Took 6.5890 seconds for prediction.
MAP:	0.019447
NDCG:	0.047864
Precision@K:	0.032007
Recall@K:	0.053703


,Dataset,Algorithm,Sparsity,Precision@k,Recall@k
0,MovieLens,BiVAE,0.900793,0.424496,0.237295
1,Book Dataset,BiVAE,0.991187,0.047196,0.067536
2,Food.com,BiVAE,0.975859,0.035045,0.039292
3,Amazon Product,BiVAE,0.991913,0.027479,0.047853
4,Ciao,BiVAE,0.991487,0.089396,0.103509
5,MovieLens,VAE,0.900793,0.386426,0.217139
6,Book Dataset,VAE,0.991187,0.028905,0.040259
7,Food.com,VAE,0.975859,0.042745,0.045760
8,Ciao,VAE,0.991487,0.089494,0.102869
9,Amazon Product,VAE,0.991913,0.032007,0.053703


In [ ]:
summary_df[['Dataset','Algorithm','Precision@k']].sort_values(by=['Dataset','Precision@k'],ascending=False)

,Dataset,Algorithm,Precision@k
0,MovieLens,BiVAE,0.424496
5,MovieLens,VAE,0.386426
7,Food.com,VAE,0.042745
2,Food.com,BiVAE,0.035045
8,Ciao,VAE,0.089494
4,Ciao,BiVAE,0.089396
1,Book Dataset,BiVAE,0.047196
6,Book Dataset,VAE,0.028905
9,Amazon Product,VAE,0.032007
3,Amazon Product,BiVAE,0.027479


In [ ]:
summary_df.to_csv('summaryVAEExplicit.csv')